## Text Features
In this practical we will focus on text data, and how we can build text processing pipelines to solve text processing problems. In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors. In this notebook, we will consider ways we can pre-process text data and engineer text features.

Specifically, we will consider: basic text data, 'bag-of-words' representations, reweighting words based on the information they contain, and basic text pipelines.

Let's begin by importing the required packages.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

### Text Data
Let us first look at some example text data. 

Run the following code to create our example data.

In [ ]:
text_data = [
    "The cat is big",
    "The dog is big",
    "The cat jumped over the dog",
    "The mouse is small",
    "The mouse jumped over the cat",
    "The cat jumped further than the mouse",
    "The ant is smallest",
    "The ant jumped over nothing",
]

The above text data consists of a list of strings. These strings can be: "documents", "reviews", "tweets", "sentences" or any other block of text. This data is in list format, and we can extract and manipulate individual text examples.

For example, the following code to print the last three characters of 5th sample.

In [ ]:
print(text_data[4][-3:])

We can also do more advanced string manipulations, like remove the word 'The' from each example.

Run the following code to print the data, but with the the word 'The' (and the subsequent space) removed from each example.

In [ ]:
print([x.replace('The ','') for x in text_data])

### Bag of Words
To perform machine learning on text documents, we need to turn the text content into numerical feature vectors. One of the most common ways to do this is to count the number of times each word occurs in each document and represent each document using the corresponding vector of counts.

We can do this using the `CountVectorizer` feature extraction method from Sklearn. Run the following code to see what `CountVectorizer` does. 

In [ ]:
# Helper function
def create_doc_feature_df(sparse_mat, feature_names):
    return(pd.DataFrame.sparse.from_spmatrix(sparse_mat, 
                        columns=feature_names))
# -----
count_vect = CountVectorizer()
count_vect.fit(text_data)
create_doc_feature_df(count_vect.transform(text_data),count_vect.get_feature_names()).head(10)

The function `create_doc_feature_df` is just a helper function to help visualise the output of ` CountVectorizer`. It creates a document-feature matrix showing values for each document-feature combination in the data.

### Stop-words
Often, many of the words in the text data will not be useful for solving the machine learning problem, and may even negatively impact performance. It is common to filter out these words.

A set of words that are often filtered out of text processing pipelines are *stop-words*, examples of which are the words 'is', 'the', 'and' etc. `CountVectorizer` makes it simple to filter out stop-words. Run the code below to create another document-feature matrix, but this time with stop-words removed.

*Note*: You may not always want to filter out stop-words.

In [ ]:
count_vect_nostops = CountVectorizer(stop_words='english')
count_vect_nostops.fit(text_data)
create_doc_feature_df(count_vect_nostops.transform(text_data),count_vect_nostops.get_feature_names()).head(10)

### N-grams
Sometime single words on there own are not enough to capture the relevant information in some input text data, and we need to consider combinations of words. For example, consider the differen between the phases 'great' and 'not great'.

Using `CountVectorizer` it is easy to create text features from sequences of words called *n-grams*. Run the following code to create a document-feature matrix that includes single words (*unigrams*) and pairs of words (*bigrams*).

In [ ]:
count_vect_bigrams = CountVectorizer(stop_words='english', ngram_range=(2, 2))
count_vect_bigrams.fit(text_data)
create_doc_feature_df(count_vect_bigrams.transform(text_data),count_vect_bigrams.get_feature_names()).head(10)

### Term Frequency–Inverse Document Frequency 
While the raw counts output from `CountVectorizer` could be inputted into a machine learning algorithm, it is common to normalise the counts to better represent the *importance* of words. For example, a word occuring in every document might not be as useful as a word that only occurs a subset of the documents.

To capture the importance of words, and down-/up-weight words that are less/more important, we apply a *term frequency–inverse document frequency* ([Tfidf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)) transformation. This can be done using the `TfidfTransformer` method from Sklearn.

Run the following code to create a document-feature matrix where the word-features have been normalised by the number of documents in the dataset that contain that word. 

*Note*: To make things simpler, will have created a `Pipeline` object. We did this because `TfidfTransformer` requires us to count the number of word occurences first using `CountVectorizer`.

In [ ]:
text_pipe = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
])
text_pipe.fit(text_data);
create_doc_feature_df(text_pipe.transform(text_data), text_pipe.named_steps['vect'].get_feature_names()).head(10)